In [ ]:
import os, sys, datetime, time, json, numpy as np
import matplotlib.pyplot as plt

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
print(f"Current path: {os.getcwd()}")
sys.path.append('./')

from supervisedlearning.homoskedastic import GaussianProcessRegressorRBF
from supervisedlearning.heteroskedastic import HeteroskedasticGaussianProcessRegressorRBF

from utils import make_json_serializable

In [ ]:
kernel_L = 10
regularization = 1
n_samples_plt = 100
x_plt = np.linspace(0, 1, n_samples_plt)

N_lst = [500, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
Z_lst = [50, 100, 500]

n_trials = 5

HET = "\\hetgp"
HOM = "\\homgp"
alg_lst = [HOM, HET]

def exp_reward_values(xvect):
    return 0.5 * np.cos(xvect * 6.28 - 1) + 0.5

elapsed_time_dict = {}

print(f"Running experiment with Z in {Z_lst} and N in {N_lst} for {n_trials} trials each")

for Z in Z_lst:

    print(f"Starting experiment with Z = {Z}")

    elapsed_time_dict[Z] = {}
    elapsed_time_dict[Z][HOM] = -1 * np.ones((len(N_lst), n_trials))
    elapsed_time_dict[Z][HET] = -1 * np.ones((len(N_lst), n_trials))

    samples_unique = np.linspace(0, 1, Z)

    for N_i, N in enumerate(N_lst):

        print(f"\tStarting experiment with N = {N}")

        x_vect = np.repeat(samples_unique, int(N/Z))
        y_vect = exp_reward_values(x_vect)

        x_vect = x_vect.reshape(N, 1)
        y_vect = y_vect.reshape(N, 1)

        for trial_i in range(n_trials):

            print(f"\t\tStarting trial {trial_i+1} of {n_trials}")

            y_hat = {}
            y_sigma = {}

            for alg in alg_lst:

                # print(f"\t\t\t{alg} starting")
                start_time = time.time()
                if alg == HOM:
                    regressor = GaussianProcessRegressorRBF(kernel_L, regularization)
                elif alg == HET:
                    regressor = HeteroskedasticGaussianProcessRegressorRBF(kernel_L, regularization)
                else:
                    raise NotImplementedError(f"{alg} not implemented yet")
                regressor.load_data(x_vect, y_vect)
                y_hat[alg], y_sigma[alg] = regressor.compute(x_plt.reshape(-1, 1))
                elapsed_time = time.time() - start_time
                elapsed_time_dict[Z][alg][N_i, trial_i] = elapsed_time
                print(f"\t\t\t{alg} completed in {np.round(elapsed_time, 2)} seconds")

            print(f"\t\t\tError - PostMean: {np.round(np.sum(np.abs(y_hat[HET] - y_hat[HOM])), 15)} - PostVariance: {np.round(np.sum(np.abs(y_sigma[HET] - y_sigma[HOM])), 15)}")

In [ ]:
dateformat = "(%Y-%b-%d %Ih%Mm%p)"
save_path = "results/supervised/runningtime/"

for Z in list(elapsed_time_dict.keys()):
    
    plt.figure()
    
    for alg in list(elapsed_time_dict[Z].keys()):

        mx = elapsed_time_dict[Z][alg]
        
        points_mean = np.mean(mx, axis=1)
        points_ci = np.std(mx, axis=1) * 1.96 / np.sqrt(n_trials)

        plt.plot(np.array(N_lst), points_mean, "o-", label=alg)
        plt.fill_between(np.array(N_lst), points_mean - points_ci, points_mean + points_ci, alpha=0.3)

    plt.legend()
    plt.xlim([0, max(N_lst) + min(N_lst)])
    
    complete_path = f"{save_path}Z{Z}_Nlst{N_lst}_trials{n_trials}_algs{alg_lst}_{datetime.datetime.now().strftime(dateformat)}"

    plt.savefig(f"{complete_path}.jpg")

    try:
        import tikzplotlib as tkz
        tkz.save(f"{complete_path}.tex")
    except Exception as e:
        print(f"Error: {e}")
        print("failed to save in tikz")

serializable_dict = make_json_serializable(elapsed_time_dict)
complete_path_json = f"{save_path}Zlst{Z_lst}_Nlst{N_lst}_trials{n_trials}_algs{alg_lst}_{datetime.datetime.now().strftime(dateformat)}.json"
with open(complete_path_json, "w", encoding="utf-8") as time_json:
    json.dump(serializable_dict, time_json, ensure_ascii=False, indent=4)